# library

In [170]:
import numpy as np
import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler

from data.load import Data
from src.models.classification.models import Model

In [171]:
%cd Credit-rating-classification

[Errno 2] No such file or directory: 'Credit-rating-classification'
/Users/kangjiwon/Desktop/HACKATHON/Credit-rating-classification


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [172]:
# !pip install -U scikit-learn
# !pip install pandas numpy xgboost

### Utils

In [227]:
def get_max_score_model_instance(model_info_list):
    max_score = model_info_list[0].get("model_score", 0)
    max_score_model_instance = model_info_list[0].get("model_instance")
    max_score_model_name = model_info_list[0].get("model_name")

    for model_info in model_info_list[1:]:
        model_score = model_info.get("model_score", 0)
        model_instance = model_info.get("model_instance")
        model_name = model_info.get("model_name")
        
        if model_score > max_score:
            max_score = model_score
            max_score_model_instance = model_instance
            max_score_model_name = model_name
        
    print(f"MAX SCORE model instance: {max_score_model_name} TEST SCORE: {max_score}")
    return max_score_model_instance

In [663]:
class ModelProcess:
    def __init__(self):
        """ 생성자 """
        self._train = Data.train
        self._test = Data.test
        self._submit = Data.submission

        self.result = None
    
    def preprocessing(self, df):
        """ 전처리기 """
        #init
        df = df.drop(columns=['ID','근로기간','연체계좌수'])

        #------------------------------------------------JW------------------------------------------------------#
        """연체 관련"""
        # index_list = df[(df['총연체금액']!=0.0) | (df['연체계좌수']!=0.0) | (df['최근_2년간_연체_횟수']!=0.0)].index
        # df.loc[index_list,'연체유무'] = 1
        # df.fillna(0, inplace=True)
        # df['연체유무'] = df['연체유무'].astype(int)
        # df.drop(['연체계좌수','총연체금액'], axis=1, inplace=True)

        # index_list = df[df['연체계좌수']!=0.0].index # 연체계좌수가 존재하면 연체 유
        # df.loc[index_list,'연체유무'] = 1
        # df.fillna(0, inplace=True)
        # df['연체유무'] = df['연체유무'].astype(int)
        # df.drop(['최근_2년간_연체_횟수','총연체금액','연체계좌수'], axis=1, inplace=True)
        
        # 최근_2년간_연체_횟수 -> 최근_2년간_연체 유무
        is_overdue = df[df['최근_2년간_연체_횟수']!=0.0].index
        df.loc[is_overdue,'최근_2년간_연체_횟수']=1.0
        df['최근_2년간_연체']=df['최근_2년간_연체_횟수']
        df.drop('최근_2년간_연체_횟수', axis=1, inplace=True)

        # '총연체금액' 범주화
        # bins = [1, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000]
        # df['총연체금액'] = pd.cut(df['총연체금액'], bins=bins, labels=False).fillna(-1.0)
                
       
        # df['대출기간'] = df['대출기간'].astype(str).str[:3].astype(int)
        # df['월원금상환금'] = df['대출금액']/df['대출기간']
        # df['상환개월'] = df['총상환원금']/df['월원금상환금']
        # df.drop(['월원금상환금'],axis=1, inplace=True)
        # df['상환개월'] = df['상환개월'].round().astype(int)

        """ 부채관련 """
        # 월원금상환금
        df['대출기간'] = df['대출기간'].astype(str).str[:3].astype(int)
        df['월원금상환금'] = round(df['대출금액']/df['대출기간'])
        # 상환개월
        df['상환개월'] = np.trunc(df['총상환원금']/df['월원금상환금'])
        
        df.drop(['월원금상환금'], axis=1, inplace=True)
        
        #------------------------------------------------------------#
        
        #sb
        # 'ANY' 컬럼 지우기
        # df[df["주택소유상태"] != "ANY"].reset_index(drop = True,inplace = True)
        # train_object = df['대출목적'].unique()
        # oject_dict = dict()
        # for i, v in enumerate(df['대출목적'].unique()):
        #     oject_dict[v] = i
        # df['대출목적'] = df['대출목적'].apply(lambda x: oject_dict.get(x, '기타'))
        #------------------------------------------------------------#
        
        #GY
        # df['연간소득'] = pd.qcut(df['연간소득'], q=10, labels = False)
        # df["총상환금"] = df["총상환원금"] + df["총상환이자"]
        # df["남은대출금액"] = df["대출금액"] - df["총상환금"]
        # df = df.drop(columns=['총상환금'])
        #------------------------------------------------------------#
        
        # label 범주형 수치형으로 변환
        categorical_features = ['대출기간', '주택소유상태', '대출목적']
        for i in categorical_features:
            le = LabelEncoder()
            df[i] = le.fit_transform(df[i])
        return df
        
    def train(self, _model):
        """ model 훈련 """
        # train 전처리
        print(f"MODE: {str(_model)}")
        self._train = self.preprocessing(self._train)
        
        # train 분리
        X = self._train.drop(['대출등급'],axis = 1)
        Y = self._train['대출등급']
        
        self.label_encoder = LabelEncoder()
        Y = self.label_encoder.fit_transform(Y)
        
        x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.1 ,random_state=42,stratify=Y)

        
        # model 훈련 및 평가
        _model.fit(X_train_scaled,y_train)
        y_pred = _model.predict(X_train_scaled)
        score = f1_score(y_train,y_pred, average="macro")
        print(f"[Train Score]: {score:.2f}", end = " ")

        y_pred = _model.predict(x_test)
        score = f1_score(y_test,y_pred, average="macro")
        print(f"[Test Score]: {score:.5f}")

        
        model_name = str(_model).split("(")[0]
        #전체 학습 시
        # _model.fit(X,Y)
        # score = 0.5
        return {
            "model_name": model_name,
            "model_instance": _model,
            "model_score": round(score, 2)
        }
        
    def test(self, _model):
        """ model 추론 """
        # test 전처리
        self._test = self.preprocessing(Data.test) 
         # 테스트 데이터의 스케일링
        # X_test_scaled = self.robust_scaler.transform(x_test)
        # self._test = self.scaler(self._test) #scaler
        print("TEST SCALE RESULT")
        print(_test)
        # model 예측
        real = _model.predict(_test)
        pred = self.label_encoder.inverse_transform(real)
        self.result = pred
        # self.result = np.where(real == 0, 'A', 
        #               np.where(real == 1, 'B',
        #               np.where(real == 2, 'C',
        #               np.where(real == 3, 'D',
        #               np.where(real == 4, 'E', 'F')))))
        return _model
        
    def submit(self, _model):
        """ 제출 """
        # csv 저장
        self._submit['대출등급'] = self.result
        self._submit.to_csv("submit_2.csv",index = False)
        
        # 모델 저장
        # model_name = str(_model).split("(")[0]
        with open('model.pkl', 'wb') as file:
            pickle.dump(_model, file)
    
    def auto_test(self):
        model_set = Model()
        models = model_set.get_model_instances()

        model_info_list = list(map(self.train, models))
        model_instance = get_max_score_model_instance(model_info_list)

        self.test(model_instance)
        self.submit(model_instance)
            

In [664]:
model_process = ModelProcess()

### 전체 모델을 대상 학습 및 테스트 평가

In [665]:
# model_process.auto_test()

### 개별 모델 학습

In [666]:
# model1 = MLPClassifier(hidden_layer_sizes=(50,50), max_iter=1000, random_state=1)
model_set = Model()
models = model_set.get_model_instances()

필요한 모듈이 설치되지 않았습니다: No module named 'catboost'
필요한 모듈이 설치되지 않았습니다: No module named 'lightgbm'


In [667]:
# model_li = list(map(model_process.train, models))

In [668]:
# 데이터 로드
with open('./data/best_params.pkl', 'rb') as f:
	data = pickle.load(f)

In [669]:
# data

In [670]:
_model = model_process.train(models[-4].set_params(**data))

MODE: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8734150979529018, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.017600210882064204,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0947263260073529,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=373, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)
[Train Score]: 0.99 [Test Score]: 0.06181


##### 부채관련

상환개월 추가: 0.81857
상환개월 추가&총상환이자 Drop: 0.66796

상환개월, 이자율 추가: 0.81857
총계좌수 Drop: 0.80624

In [ ]:
# 연체계좌수 Drop 가정

# 총연체금액 Drop 0.80853
# 2년 연체 Drop 0.81141

# 총연체금액 범주화&2년 연체 Drop 0.80918
# 총연체금액 범주화&2년 연체 살리기 0.80738

# 최근_2년간_연체_횟수->최근_2년간_연체 유무 & 총연체금액 Drop 0.80649
# 최근_2년간_연체_횟수->최근_2년간_연체 유무 & 총연체금액 살리기 0.80967 ✔️

# 최근_2년간_연체_횟수->최근_2년간_연체 유무 & 총연체금액 범주화 0.80830

# ALL Drop 0.81499

In [409]:
model_process.test(_model.get("model_instance"))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8734150979529018, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.017600210882064204,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0947263260073529,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=373, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [410]:
model_process.submit(_model.get("model_instance"))

In [411]:
#0.827

#0.83033 연체계좌수
#0.82653 총연체
#0.82698 최근_2년간_연체_횟수

# 0.83029 연체계좌수
# 0.82 ~ 총연체금액
# 0.83300 최근_2년간_연체_횟수

# 0.80853
# 0.80835
# ,'총연체금액'